In [1]:
def read_input_file(file_path, size=12):

    if isinstance(file_path, str):
        file =  open(file_path, "r")
        input_data = file.readlines()
    else:
        input_data = file_path

    my_move = input_data[0].strip('\n')
    time_limits = input_data[1].strip('\n')
    my_time, opp_time = float(time_limits.split(' ')[0]), float(time_limits.split(' ')[1])

    board_state = []
    board_rows = input_data[2:]
    for row in board_rows:
        board_state.append(list(row[:-1]))

    coordinates = {'X': [], 'O': [], '.': []}
    for row_no in range(size):
        for col_no in range(size):
            if board_state[row_no][col_no] in coordinates:
                coordinates[board_state[row_no][col_no]].append((row_no, col_no))

    return my_move, my_time, opp_time, board_state, coordinates

In [2]:
def find_valid_moves(board, player, size=12):
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    opponent = 'O' if player == 'X' else 'X'
    valid_moves = []

    for i in range(size):
        for j in range(size):
            if board[i][j] != '.':
                continue  # Skip if the tile is not empty

            for dx, dy in directions:
                x, y = i + dx, j + dy

                if x < 0 or x >= size or y < 0 or y >= size or board[x][y] != opponent:
                    continue  # Skip if out of bounds or not adjacent to an opponent's piece

                # Move in the direction while it's the opponent's piece
                while 0 <= x < size and 0 <= y < size and board[x][y] == opponent:
                    x += dx
                    y += dy

                # Check if we ended up in a valid position
                if 0 <= x < size and 0 <= y < size and board[x][y] == player and (x - dx != i or y - dy != j):
                    # if (i,j) not in valid_moves:
                    #     valid_moves.append((i, j))
                    valid_moves.append((i, j))
                    break  # No need to check other directions for this position

    return valid_moves

In [3]:
def heuristic_evaluation(board, player):
    opponent = 'O' if player == 'X' else 'X'
    player_tiles = sum(row.count(player) for row in board)
    opponent_tiles = sum(row.count(opponent) for row in board)

    # Assuming 'board' is the current board state and 'player' is the current player
    stability_score = edge_stability_heuristic(board, player)

    return player_tiles - opponent_tiles + stability_score

def is_stable_disc(board, x, y, player):
    if board[x][y] != player:
        return False  # The disc is not the player's disc

    # Check if the disc is in the corner
    if (x == 0 or x == len(board) - 1) and (y == 0 or y == len(board[0]) - 1):
        return True

    # Check if the disc is on an edge and has a line of the same color discs ending in a corner
    if x == 0 or x == len(board) - 1:  # Top or bottom edge
        if board[0][y] == player and board[len(board) - 1][y] == player:
            return True
    if y == 0 or y == len(board[0]) - 1:  # Left or right edge
        if board[x][0] == player and board[x][len(board[0]) - 1] == player:
            return True

    # Check rows and columns for stability
    row_stable = all(board[i][y] == player for i in range(len(board)))
    col_stable = all(board[x][j] == player for j in range(len(board[0])))

    return row_stable or col_stable


def edge_stability_heuristic(board, player):
    opponent = 'O' if player == 'X' else 'X'
    stability_score = 0

    # Evaluate all edge positions for stability
    for x in [0, len(board) - 1]:  # Top and bottom edges
        for y in range(len(board[0])):
            if is_stable_disc(board, x, y, player):
                stability_score += 1
            elif is_stable_disc(board, x, y, opponent):
                stability_score -= 1

    for y in [0, len(board[0]) - 1]:  # Left and right edges
        for x in range(len(board)):
            if is_stable_disc(board, x, y, player):
                stability_score += 1
            elif is_stable_disc(board, x, y, opponent):
                stability_score -= 1

    return stability_score

In [4]:
def minimax(board, depth, alpha, beta, maximizingPlayer, player):
    if depth == 0 or is_terminal_node(board):
        return heuristic_evaluation(board, player)

    valid_moves = find_valid_moves(board, player if maximizingPlayer else 'O' if player == 'X' else 'X', size=12)

    if not valid_moves:
        return heuristic_evaluation(board, player)

    if maximizingPlayer:
        maxEval = float('-inf')
        for move in valid_moves:
            new_board = make_move(board, move, player)
            eval = minimax(new_board, depth - 1, alpha, beta, False, player)
            maxEval = max(maxEval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval
    else:
        minEval = float('inf')
        for move in valid_moves:
            new_board = make_move(board, move, 'O' if player == 'X' else 'X')
            eval = minimax(new_board, depth - 1, alpha, beta, True, player)
            minEval = min(minEval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval

In [5]:
def make_move(board, move, player):
    # Deep copy the board to avoid modifying the original
    new_board = [row[:] for row in board]

    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    opponent = 'O' if player == 'X' else 'X'
    x, y = move

    # Place the player's piece on the board
    new_board[x][y] = player

    # For each direction, look for opponent pieces to flip
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        pieces_to_flip = []

        while 0 <= nx < len(new_board) and 0 <= ny < len(new_board[0]) and new_board[nx][ny] == opponent:
            pieces_to_flip.append((nx, ny))
            nx += dx
            ny += dy

        # Check if the line ends with a player's piece to validate flipping
        if 0 <= nx < len(new_board) and 0 <= ny < len(new_board[0]) and new_board[nx][ny] == player:
            for px, py in pieces_to_flip:
                new_board[px][py] = player

    return new_board

In [6]:
def is_terminal_node(board):
    if find_valid_moves(board, 'X', size=12) or find_valid_moves(board, 'O', size=12):
        return False
    return True

In [7]:
def choose_best_move_minimax(board, player, depth=5):
    best_score = float('-inf')
    best_move = None
    valid_moves = find_valid_moves(board, player, size=12)

    corners = [(0,0), (11,0), (0,11), (11,11)]
    valid_corners = []
    for move in corners:
        if move in valid_moves:
            valid_corners.append(move)
    if valid_corners:
        valid_moves = valid_corners

    for move in valid_moves:
        new_board = make_move(board, move, player)
        score = minimax(new_board, depth, float('-inf'), float('inf'), False, player)
        if score > best_score:
            best_score = score
            best_move = move

    if best_move:
        col_nos = [chr(i) for i in range(ord('a'), ord('l')+1)]
        col_no = col_nos[best_move[1]]
        row_no = best_move[0] + 1
        return f"{col_no}{row_no}"
    return 'No move found'

In [11]:
# input_data_path = 'input.txt'
input_data_path = '/content/drive/MyDrive/USC_CS_AI/CS561_AI/HW/HW2_Game/resource/asnlib/publicdata/training/input5.txt'


my_move, my_time, opp_time, board_state, coordinates = read_input_file(input_data_path)
if my_time>299:depth=2
elif my_time>210:depth=5
elif my_time>165:depth=4
else:depth=3

next_move = choose_best_move_minimax(board_state, my_move, depth=5)
print(next_move)

# f = open('output.txt', 'w')
# f.write(next_move+'\n')
# f.close()

g6
